# **Tarefa 1**

In [ ]:
!pip install pandas
import pandas as pd
pd.set_option('display.max_colwidth', None)

df = pd.read_parquet("hf://datasets/tatsu-lab/alpaca/data/train-00000-of-00001-a09b74b3ef9c3b56.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install scikit-learn
from sklearn.model_selection import train_test_split

#Configura o tamanho da amostra
df_sample = df.sample(n=6000, replace=True, random_state=42)

#Divide entre dados de treinamento e teste
train_df, test_df = train_test_split(df_sample, test_size=0.5)

print(train_df.shape)
print(test_df.shape)

(3000, 4)
(3000, 4)


In [ ]:
from transformers import pipeline, set_seed
# instacia o pad_token_id e eos_token_id para evitar "warnings"
generator = pipeline('text-generation', model='gpt2', pad_token_id=50256, eos_token_id=50256) # <|endoftext|> id = 50256
set_seed(42)

test_df['instruction'].head()

# Considerando que estamos fornecendo instruções para o modelo, as saídas foram configuradas com "temperature=0.5" afim de torná-lo mais determinístico e focado, como uma resposta de instrução exige.
def text_generator_small(df):
    output = [generator(x, max_new_tokens=60, temperature=0.5, top_p=0.9, num_return_sequences=1)[0]['generated_text'] for x in df['instruction']]
    df['generated'] = [gen[len(inst):].strip() for inst, gen in zip(df['instruction'], output)]
    return df

text_generator_small(test_df)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,generated
41846,"The first is the ""brain"" of AI. It is the most basic of all the disciplines of AI. It is the most basic of all the disciplines of AI. It is the most basic of all the disciplines of AI.\n\nThe second is the ""brain"" of AI."
19194,"The first character to be drawn is the name of the character, the second is the name of the character's mother, the third is the name of the character's father, and the fourth is the name of the character's mother.\n\nThe first character to be drawn is the name"
34084,Create a new topic and add a new topic to the topic.\n\nCreate a new topic and add a new topic to the topic. Create a new topic and add a new topic to the topic. Create a new topic and add a new topic to the topic. Create a new topic
40059,"3. Design and implement the project\n\nOnce you have the project, you can create a new project in Visual Studio. You can then use the Visual Studio project manager to create new projects.\n\nCreate a new project using the project manager.\n\nCreate a new project using the"
6721,"The question is, why is there such a thing as a bounded rationality?\n\nA bounded rationality is the belief that the world is finite, and that it is finite in its infinite dimension. It is not a belief that there is a finite world. It is a belief that there is"


In [ ]:
# Qualitative analysis purpose
test_df.to_csv('data_analysis.csv', sep=';', encoding='utf-8-sig', index=False)

**Avalie a qualidade do resultado. As respostas estavam corretas?**

Os resultados são diversos, de modo geral podemos destacar alguns pontos:

*   **Positivo**: as respostas são gramaticalmente corretas e seguem uma estrutura morfossintática adequada para a formação das frases.
*  **Negativo**: respostas vagas, fora do escopo da instrução, repetitivas ou incompletas.

Alguns padrões reconhecidos via análise qualitativa das respostas:
1.   Repetição de informação ou prolixia afim de ocupar todos o cumprimento de tokens máximo, tendo em vista que o GPT-2 nem sempre gera o <|endoftext|> por conta própria, mesmo explicitando o eos_token.
2.   Algumas das respostas consideradas corretas não focam completamente na instrução e acabam ficando inconclusivas.

**A observação principal é que parece que o modelo não foi feito para seguir instruções, mas sim gerar textos gerais.**


**Quais métodos podem ser usados para melhorá-las?**

Existem diferentes métodos que podemos utilizar para melhorar os resultados, dos mais simples aos mais complexos:

1.  Podemos dar exemplos de instrução via prompt, deixando de usar o zero-shot e usar as outras opções como one-shot ou few-shot.
2.  Para esse exemplo em específico, poderiamos adicionar os "inputs" dos exemplos que contém essa informação à "instruction" o que melhoraria a resposta dos respectivos exemplos.
3. Usar explicitamente um token de para poderia melhorar a objetividade das respostas.
4. Use de técnicas avançadas de engenharia de prompts como chain of thoghts ou reflexive promptings por meio da explicitação de intenção e raciocínio.
5. Outra forma de aprendizagem, mais complexa, é a utilização do fine-tuning, usando modelos maiores ou mais específico para a tarefa determinada.



# **Tarefa 2**

In [ ]:
!pip install transformers datasets accelerate
from transformers import GPT2Tokenizer
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token  # GPT2 não tem pad_token, usa-se eos_token


def tokenize(df):
    tokens = tokenizer(df["text"], truncation=True, padding="max_length", max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()  # supervisiona a saída inteira
    return tokens

train_dataset = train_dataset.map(tokenize, batched=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))  # inclui o pad_token


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 1024)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gpt2-alpaca-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=200,
    save_total_limit=1,
    fp16=True,  # ativar se estiver usando GPU com suporte
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()


model.save_pretrained("./gpt2-alpaca-finetuned")
tokenizer.save_pretrained("./gpt2-alpaca-finetuned")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.818000
100,0.679800
150,0.662700
200,0.617800
250,0.567900
300,0.541700
350,0.567100
400,0.546000
450,0.493800
500,0.509400


('./gpt2-alpaca-finetuned/tokenizer_config.json',
 './gpt2-alpaca-finetuned/special_tokens_map.json',
 './gpt2-alpaca-finetuned/vocab.json',
 './gpt2-alpaca-finetuned/merges.txt',
 './gpt2-alpaca-finetuned/added_tokens.json')

**Considerações**:
Para o fine-tuning utizei o modelo gpt2-medium com um conjunto maior de parâmetros, garantindo mais representações e conteúdo. Mas considerando o consumo de memória, utilizei o Trainer da Hugging Face devido sua eficiência e sinergia com os outros métodos da HF como Dataset, Tokenizer. A classe facilita na hora da execução do treinamento, trabalhando a paralezação dos dados de entrada com o uso do .map, evitando os loops de treinamento. Além de treinar, avalia e salva o modelo retreinado (fine-tuning).


# ***Tarefa 3***

In [ ]:
from transformers import pipeline, set_seed
generator_finetuned = pipeline("text-generation", model="./gpt2-alpaca-finetuned", tokenizer=tokenizer, pad_token_id=50256)
set_seed(42)

def text_generator(df):
    output = [generator_finetuned(x, max_new_tokens=60, temperature=0.5, top_p=0.9, num_return_sequences=1)[0]["generated_text"] for x in df["instruction"]]
    df["generated_FT"] = [gen[len(inst):].strip() for inst, gen in zip(df["instruction"], output)]
    return df

text_generator(test_df)


Device set to use cuda:0


,instruction,input,output,text,generated,generated_FT
41846,Compare and contrast two different disciplines within AI.,,"Machine learning and natural language processing are two commonly used disciplines of AI. Machine learning algorithms are used to analyze large datasets and identify patterns in the data, while natural language processing focuses on understanding and manipulating human language using algorithms. Machine learning algorithms use mathematical models to analyze data and make predictions, while natural language processing algorithms often use rules and semantic analysis to interpret and act on natural language input. Both approaches are used to solve a variety of problems in AI applications such as speech recognition, computer vision, and robotic control.","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCompare and contrast two different disciplines within AI.\n\n### Response:\nMachine learning and natural language processing are two commonly used disciplines of AI. Machine learning algorithms are used to analyze large datasets and identify patterns in the data, while natural language processing focuses on understanding and manipulating human language using algorithms. Machine learning algorithms use mathematical models to analyze data and make predictions, while natural language processing algorithms often use rules and semantic analysis to interpret and act on natural language input. Both approaches are used to solve a variety of problems in AI applications such as speech recognition, computer vision, and robotic control.","The first is the ""brain"" of AI. It is the most basic of all the disciplines of AI. It is the most basic of all the disciplines of AI. It is the most basic of all the disciplines of AI.\n\nThe second is the ""brain"" of AI.","AI is a field of study that focuses on the development and application of computer systems to solve complex problems, such as natural language processing, robotics, and virtual assistants. AI is also a field that involves the development of new technologies and applications, such as robotics and artificial intelligence. Both disciplines are developing rapidly"
19194,Create a character sketch for a character named Jack using only three adjectives.,,"Jack is an ambitious, witty, and dependable person.","Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a character sketch for a character named Jack using only three adjectives.\n\n### Response:\nJack is an ambitious, witty, and dependable person.","The first character to be drawn is the name of the character, the second is the name of the character's mother, the third is the name of the character's father, and the fourth is the name of the character's mother.\n\nThe first character to be drawn is the name","Write a sentence that appropriately completes the description.\n\n### Instruction:\nGenerate a story for a character named Jack that takes place in a small town.\n\n### Response:\nJack was a young man who lived in a small town. He had always been curious and adventurous, and he"
34084,Generate 5 new product ideas that could be used as a solution for the given issue.,"The issue is that people frequently discard their electronics, such as computers and phones, when they are still functioning, instead of donating or recycling them.",1. A service that provides a virtual marketplace for the donation or resale of used electronics.\n2. An app that rewards people for donating or recycling their electronics.\n3. A subscription service that includes repairs and updates for owned electronics.\n4. A program that provides free or discounted used electronics to underserved communities. \n5. An online tool that helps people repurpose their old electronics for creative activities or projects.,"Below is an instruction that describes a task, paired with an input that provides f

In [ ]:
# Para facilitar a análise qualitativa
test_df.to_csv('data_analysis_finetuning.csv', sep=';', encoding='utf-8-sig', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
import pandas as pd

df_one = pd.read_csv('/content/data_analysis.csv', sep=';')
df_three = pd.read_csv('/content/data_analysis_finetuning.csv', sep=';')

In [20]:
#!pip install evaluate
#!pip install rouge_score
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")


references = df_one["output"].tolist()
prediction_one = df_one["generated"].tolist()
prediction_three = df_three["generated_FT"].tolist()

bleu_result_one = bleu.compute(predictions=prediction_one, references=[[ref] for ref in references])
print("BLEU_One:", bleu_result_one["bleu"])
bleu_result_three = bleu.compute(predictions=prediction_three, references=[[ref] for ref in references])
print("BLEU_Three:", bleu_result_three["bleu"])

rouge_result = rouge.compute(predictions=prediction_one, references=references)
print("ROUGE-L_One:", rouge_result["rougeL"])
rouge_result = rouge.compute(predictions=prediction_three, references=references)
print("ROUGE-L_Three:", rouge_result["rougeL"])

BLEU_One: 0.02648606141040323
BLEU_Three: 0.03930300627695892
ROUGE-L_One: 0.11554130916264416
ROUGE-L_Three: 0.14000313516776325


**Interprete os resultados em comparação
com os obtidos na tarefa 1.**
No geral, considerando uma análise qualitativa, os resultados obtidos foram bem melhores, com mais contexto. OS resultados são mais satisfatórios, no qual as respostas estão quase sempre relacionadas à intrução. Além disso, o problema de repetição foi solucionado, bem como a parada da resposta, finalizando as respostas de forma correta. As respostas incorretas ocorrem, majoriaterimante, devido a não utilização do "input" como dado de entrada juntamente com a "instruction".

Em uma análise qualitativa, usamos o BLUE para avaliação da similaridade textual baseada em n-gram.
Considerando os valores de BLEU, a saída do modelo otimizado obteve uma métrica um pouco melhor. Considerando Rougue, o modelo otimizado também tem uma pequena melhora.

**OBS:** apesar das métricas estarem baixas, as respostas do modelo otimizado são bem melhores que a do modelo simples.

**Se você precisasse agrupar as perguntas
que tratam de assuntos semelhantes nos 2
datasets, como faria?**

eu utilizaria técnicas de representação semântica ou reconhecimento de caracteristicas de texto como o *tf-idf* ou *embeddings* e avaliação das medidas de similaridade como a *similaridade do cosseno*.